## Migrador instalaciones régimen especial

In [1]:
import pandas as pd
from erppeek import Client
from datetime import datetime
from tqdm import tqdm

# conexión
try:
    # Due: https://www.python.org/dev/peps/pep-0476
    import ssl
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        # Legacy Python that doesn't verify HTTPS certificates by default
        pass
    else:
        # Handle target environment that doesn't support HTTPS verification
        ssl._create_default_https_context = _create_unverified_https_context
except ImportError:
    pass

# client local
# -m giscedata_re -m giscedata_polissa -m giscedata_facturacio_distri -m giscedata_telemesura
c = Client('http://localhost:8069', 'test', 'admin', 'admin')

# TODO: informar provincia y municipi
# migrar
ret = migrator(c, 
             'UPRSL.txt', # UPRS
             'UFIUPR.txt', # UFIUPR
             'CILDATOS.txt', # CILDATOS
             'TELEMEDIDA.xlsx', # Relación cups - contador
             'XX', # provincia
             'XXXXX') # municipi
ret

In [129]:
def migrator(c, uprsl_name=None, ufiupr_name=None, cildatos_name=None, contadores_name=None, provincia=None, municipi=None):
    # objects
    partner_obj = c.model('res.partner')
    partner_addr = c.model('res.partner.address')
    instalacion_re_obj = c.model('giscedata.re')
    unidad_programacion_obj = c.model('giscedata.re.uprs')
    cups_obj = c.model('giscedata.cups.ps')
    municipi_obj = c.model('res.municipi')
    polissa_obj = c.model('giscedata.polissa')
    meter_obj = c.model('giscedata.lectures.comptador')
    tarif_obj = c.model('giscedata.polissa.tarifa')

    # df unidades de programación
    df_unidades_de_programacion = pd.read_csv(uprsl_name, 
                      sep = ';',
                     names = ['codigo', 
                            'CIF', 
                            'representante', 
                            'rel_upr_2', 
                            'name_upr', 
                            'desc_upr', 
                            'cod_participante', 
                            'tarifa_o_mercado',
                            'tecnologia', 
                            'exenta_pago', 
                            'data_alta',
                            'data_baixa', 
                            '_'], 
                            dtype={'data_alta': str, 'data_baixa': str})
    
    df_unidades_de_programacion.data_alta = df_unidades_de_programacion.data_alta.apply(
        lambda x: '{0}-{1}-{2} 00:00:00'.format(x[:4], x[4:6], x[6:8]))
    df_unidades_de_programacion['data_baixa'] = df_unidades_de_programacion.data_baixa.apply(
        lambda x: '{0}-{1}-{2} 00:00:00'.format(x[:4], x[4:6], x[6:8]))
    df_unidades_de_programacion['cod_participante'] = df_unidades_de_programacion.cod_participante.apply(
        lambda x: str(x).zfill(4))

    # df instalaciones re
    df_instalaciones_re = pd.read_csv(ufiupr_name, 
                     sep = ';', 
                     names = ['codigo_unidad_fisica', 
                            'name_upr', 
                            'name',
                            'descripcio', 
                            'raipre', 
                            'cups', 
                            'cups2', 
                            'data_inici', 
                            'data_final', 
                            '_'])
    df_instalaciones_re['cups'] = df_instalaciones_re['cups'].apply(lambda x: x[:-2])

    # df cildatos (tipo_punto_frontera)
    df_cildatos = pd.read_csv(cildatos_name, 
                     sep = ';',
                     names = ['cups', 
                            'minetad', 
                            'num_reg_provisional', 
                            'num_reg_definitiva', 
                            'cod_postal', 
                            'potencia_nominal', 
                            'nombre_instalacion', 
                            'psse',
                            'grupo_rd', 
                            'tipo_punto_fontera', 
                            'fecha_alta',
                            'fecha_baja',
                            'tensión_conexión',
                            'fecha_acta_alta',
                            'propiedad_equipo_medida',
                            '_'],
                             dtype={'tipo_punto_fontera': str, 'potencia_nominal': float})
    df_cildatos['cups'] = df_cildatos['cups'].apply(lambda x: x[:-3])
    
    # df contadores
    df_contadores = pd.read_excel(contadores_name,
                                 names = ['idpm', 
                                        'name_reg', 
                                        'cups', 
                                        'phone', 
                                        'meter_name', 
                                        'link', 
                                        'dir', 
                                        'key_access',
                                        'type_point', 
                                        'notes'],
                                 dtype={'meter_name': str})
    
    # merge dfs
    df_instalaciones_re = pd.merge(df_unidades_de_programacion, df_instalaciones_re, on='name_upr', how='right')
    df_instalaciones_re = pd.merge(df_contadores, df_instalaciones_re, on='cups', how='right')
     df_instalaciones_re = pd.merge(df_cildatos, df_instalaciones_re, on='cups', how='right')
    df_instalaciones_re = df_instalaciones_re[['CIF', 
                                               'representante', 
                                               'name_upr', 
                                               'cod_participante', 
                                               'cups', 
                                               'data_alta', 
                                               'data_baixa', 
                                               'name',
                                               'meter_name',
                                               'tipo_punto_fontera',
                                               'potencia_nominal',
                                              ]]

    # TODO: DEFAULT instalaciones re
    df_instalaciones_re['provincia'] = provincia
    df_instalaciones_re['municipi_name'] = municipi

    # migrar datos
    for instalacion_re_data in tqdm(df_instalaciones_re.T.to_dict().values()):

        # buscar titular para la instalacion re
        titular_name = instalacion_re_data['name']
        titular_ids = partner_obj.search([('name', '=', titular_name)])
        if not titular_ids:
            # crear titular
            titular_id = partner_obj.create({'name': titular_name, 'vat': ''})
            titular_id = titular_id.id
            p_address_id = partner_addr.create({'name': titular_name, 'partner_id': titular_id})
            print('Titular creat: {}'.format(titular_name))
        else:
            titular_id = titular_ids[0]
        
        # buscar cups para la instalacion re
        cups_name = instalacion_re_data['cups']
        cups_ids = cups_obj.search([('name', '=', cups_name)])
        if not cups_ids:
            # crear cups
            municipi_name = instalacion_re_data['municipi_name']
            municipi_ids = municipi_obj.search([('name', '=', municipi_name)])
            cups_id = cups_obj.create({'name': cups_name, 'id_municipi': municipi_ids[0]})
            # raise Exception('Error: el cups {} no existe. Manual Create...'.format(cups_name))
        else:
            cups_id = cups_ids[0]
        
        # buscar el contrato del cups
        contrato_ids = polissa_obj.search([('cups', '=', cups_name)])
        if not contrato_ids:
            # crear contrato
            tariff_id = tarif_obj.search([('name', '=', 'RE')])[0]
            p_id = partner_obj.search([('ref', '=', instalacion_re_data['cod_participante'])])
            if p_id:
                p_id = p_id[0]
            else:
                p_id = False
                
            contrato_id = polissa_obj.create(
                {
                    'tarifa': tariff_id, 
                    'cups': cups_id, 
                    'titular': titular_id,
                    'data_alta': instalacion_re_data['data_alta'],
                    'comercialitzadora': p_id
                }
            )
        else:
            contrato_id = contrato_ids[0]
        
        # buscar el contador
        meter_ids = meter_obj.search([('name', '=', instalacion_re_data['meter_name'])])
        if not meter_ids:
            reg_obj = c.GiscedataRegistrador
            reg_id = reg_obj.search(
                [('name', '=', instalacion_re_data['meter_name']), 
                 ('type', '=', 'billing'), 
                 ('technology', '=', 'telemeasure')]
            )
            # relacionar contador con el registrador
            if reg_id:
                # el registrador ya existe
                reg_id = reg_id[0]
            else:
                # creamos el registrador
                reg_id = reg_obj.create(
                    {
                        'name': instalacion_re_data['meter_name'], 
                        'type': 'billing', 
                        'technology': 'telemeasure'
                    }
                )
                reg_id = reg_id.id
            # crear contador
            meter_id = meter_obj.create(
                {
                    'name': instalacion_re_data['meter_name'],
                    'meter_type': 'G',
                    'data_alta': instalacion_re_data['data_alta'],
                    'polissa': contrato_id,
                    'technology_type': 'telemeasure',
                    'bloquejar_cini': True,
                    'registrador_id': reg_id
                }
            )

        # crear instalacion_re
        instalacion_re_id = instalacion_re_obj.create(
            {
                'name': instalacion_re_data['name'],
                'cups': cups_id,
                'titular': titular_id,
                'provincia': instalacion_re_data['provincia'],
                'tipo': instalacion_re_data['tipo_punto_fontera'],
                'potencia_nominal': instalacion_re_data['potencia_nominal']
            }
        )

        # UPRS
        representantes_ids = partner_obj.search([('name', '=', instalacion_re_data['representante'])])
        if not representantes_ids:
            representantes_ids = partner_obj.create(
                {
                    'name': instalacion_re_data['representante'], 
                    'ref': instalacion_re_data['cod_participante'],
                    'vat': 'ES' + instalacion_re_data['CIF'],
                }
            )
            representantes_ids = representantes_ids.id
            p_address_id = partner_addr.create({'name': instalacion_re_data['representante'], 'partner_id': representantes_ids})
            print('Representante creado. Validar datos: {}'.format(instalacion_re_data['representante']))
        else:
            representantes_ids = representantes_ids[0]
            ref = partner_obj.read(representantes_ids, ['ref'])['ref']
            if not ref:
                partner_obj.write(representantes_ids, {'ref': instalacion_re_data['cod_participante']})
        unidad_programacion_obj.create(
            {
                'name': instalacion_re_data['name_upr'], 
                'representante': representantes_ids, 
                'inst_re': instalacion_re_id.id,
                'data_alta': instalacion_re_data['data_alta'],
                'data_baixa': instalacion_re_data['data_baixa'],
            }
        )
    
    return df_instalaciones_re